In [ ]:
%load_ext dotenv
%dotenv

In [ ]:
import ast
from pathlib import Path
import sqlite3

import fsspec
import pandas as pd
from tqdm.notebook import tqdm_notebook

In [ ]:
fs = fsspec.filesystem("abfs")
CONTAINER = "team1-3"
fs.ls(CONTAINER)

In [ ]:
def path_blob(path):
    return f"{CONTAINER}/{path}"

In [ ]:
path_db = "April9.sqlite"
Path(path_db).unlink(missing_ok=True)

---

# Transcoding the April 9 data dump to a SQLite database

tl;dr: just fetch the file off the Azure container

In [ ]:
fs.get(path_blob(path_db), path_db)

---

## Doing the transcode boogie

In [ ]:
def stream_beacons(path):
    with fs.open(path_blob(path), mode="r", encoding="utf-8") as file:
        for line in file:
            yield ast.literal_eval(line)

In [ ]:
num_beacons = sum([1 for beacon in tqdm_notebook(stream_beacons("April9DataOnly.txt"))])
num_beacons

In [ ]:
with sqlite3.connect(path_db) as db:
    db.execute("CREATE TABLE IF NOT EXISTS beacons(timestamp, manufacturerDataHex, rssi)")
    for beacon in tqdm_notebook(stream_beacons("April9DataOnly.txt"), total=num_beacons):
        db.execute(
            "INSERT INTO beacons(manufacturerDataHex, rssi) VALUES (?, ?)",
            (beacon["manufacturerDataHex"], beacon["rssi"])
        )

In [ ]:
with sqlite3.connect(path_db) as db:
    cursor = db.cursor()
    cursor.execute(
        'SELECT rssi FROM beacons WHERE manufacturerDataHex = ?',
        ("4c00121900d27e0900d6ca0770107d075fbe7dd7e98b6692dd4f290000",)
    )
    for i, (rssi,) in enumerate(cursor):
        if i >= 5:
            break
        print(rssi)

🍰

---

To upload the SQLite file:

In [ ]:
fs.put(str(path_db), path_blob(path_db))

In [ ]:
fs.ls(CONTAINER)